In [1]:
import torch
from torchvision import datasets, transforms
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

In [2]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)),
                                transforms.Resize(224),
                                transforms.CenterCrop(224),
                                ])

### Standard dataset

In [3]:
trainset = datasets.CIFAR10(root="cifar10", download=False, train=True, transform=transform)
testset = datasets.CIFAR10(root="cifar10", download=True, train=False, transform=transform)

Files already downloaded and verified


### Custom dataset

In [4]:
trainset = datasets.ImageFolder("/home/dell/Python_Files/Celebrities/training_set",transform=transform)
testset = datasets.ImageFolder("/home/dell/Python_Files/Celebrities/test_set",transform=transform)

In [5]:
split_size = 0.2
num_train = len(trainset)
indices = [i for i in range(num_train)]
num_val = int(split_size * num_train)
train_idx, valid_idx = indices[num_val:], indices[:num_val]

In [6]:
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

In [7]:
train_generator = DataLoader(trainset, batch_size=10, sampler=train_sampler)
val_generator = DataLoader(trainset, batch_size=5, sampler=valid_sampler)
test_generator = DataLoader(testset, batch_size=5)

In [9]:
dataiterator = iter(train_generator)
images, labels = next(dataiterator)
print(images.shape)
print(labels)

torch.Size([10, 3, 224, 224])
tensor([4, 3, 4, 4, 4, 3, 4, 4, 1, 4])


In [10]:
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,3,padding=1)
        self.conv2 = nn.Conv2d(16,16,3,padding=1)
        self.conv3 = nn.Conv2d(16,32,3,padding=1)
        self.conv4 = nn.Conv2d(32,64,3,padding=1)
        self.conv5 = nn.Conv2d(64,128,3,padding=1)
            
        self.pool = nn.MaxPool2d((2,2))
            
        self.hidden1 = nn.Linear(7*7*128, 128)
        self.hidden2 = nn.Linear(128, 256)
        self.hidden3 = nn.Linear(256, 512)
        self.output = nn.Linear(512,5)
            
        self.dropout = nn.Dropout(0.2)
            
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.conv5(x))
        x = self.pool(x)
            
        x = x.view(-1, 7*7*128)
            
        x = F.relu(self.hidden1(x))
        x = self.dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.dropout(x)
        x = F.relu(self.hidden3(x))
        x = self.dropout(x)
        x = self.output(x)
            
        return x

In [16]:
!pip install torchsummary

  Using cached torchsummary-1.5.1-py3-none-any.whl (2.8 kB)


In [19]:
from torchsummary import summary
model = Classifier()
summary(model, input_size=(3, 224, 224))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            448
├─Conv2d: 1-2                            2,320
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            18,496
├─Conv2d: 1-5                            73,856
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            802,944
├─Linear: 1-8                            33,024
├─Linear: 1-9                            131,584
├─Linear: 1-10                           2,565
├─Dropout: 1-11                          --
Total params: 1,069,877
Trainable params: 1,069,877
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            448
├─Conv2d: 1-2                            2,320
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            18,496
├─Conv2d: 1-5                            73,856
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            802,944
├─Linear: 1-8                            33,024
├─Linear: 1-9                            131,584
├─Linear: 1-10                           2,565
├─Dropout: 1-11                          --
Total params: 1,069,877
Trainable params: 1,069,877
Non-trainable params: 0

In [20]:
criterion = nn.CrossEntropyLoss()

In [21]:
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [31]:
epochs = 1
for i in range(epochs):
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    for images, labels in train_generator:
        #print(images)
        #print(labels)
        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        #print(loss.item())
        train_loss += loss.item()*images.size(0)
        #print(train_loss)
    
    model.eval()
    for images, labels in val_generator:
        optimizer.zero_grad()
        pred = model(images)
        loss = criterion(pred, labels)
        val_loss += loss.item()*images.size(0)
        #print(val_loss)
#print(len(train_generator))
#print(len(val_generator))
train_loss = train_loss/len(train_generator)
val_loss = val_loss/len(val_generator)
print(train_loss)
print(val_loss)

13.995614647865295
43.66772413253784


In [34]:
test_accuracy = 0.0
for images, labels in test_generator:
    output = model(images)
    #print(output)
    _, pred = torch.max(output,1)
    #print(_)
    #print(pred)
    #print(list(pred))
    print(labels)
    num_correct = 0
    accuracy = 0.0
    for i in range(len(pred)):
        if pred[i]==labels[i]:
            num_correct+=1
    accuracy = (num_correct/images.size(0))*100
    #print(accuracy)
    test_accuracy += accuracy
#print(test_accuracy)
print(test_accuracy/len(test_generator))

tensor([0, 0, 0, 0, 0])
tensor([1, 1, 1, 1, 1])
tensor([2, 2, 2, 2, 2])
tensor([3, 3, 3, 3, 3])
tensor([4, 4, 4, 4, 4])
20.0


### Saving and Loading models

In [37]:
torch.save(model.state_dict(), "celebrity_pytorch.pt")

In [38]:
model.load_state_dict(torch.load("celebrity_pytorch.pt"))

<All keys matched successfully>

In [39]:
summary(model, input_size=(3,224,224))

Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            448
├─Conv2d: 1-2                            2,320
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            18,496
├─Conv2d: 1-5                            73,856
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            802,944
├─Linear: 1-8                            33,024
├─Linear: 1-9                            131,584
├─Linear: 1-10                           2,565
├─Dropout: 1-11                          --
Total params: 1,069,877
Trainable params: 1,069,877
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            448
├─Conv2d: 1-2                            2,320
├─Conv2d: 1-3                            4,640
├─Conv2d: 1-4                            18,496
├─Conv2d: 1-5                            73,856
├─MaxPool2d: 1-6                         --
├─Linear: 1-7                            802,944
├─Linear: 1-8                            33,024
├─Linear: 1-9                            131,584
├─Linear: 1-10                           2,565
├─Dropout: 1-11                          --
Total params: 1,069,877
Trainable params: 1,069,877
Non-trainable params: 0

### Pretrained models

In [42]:
from torchsummary import summary
from torchvision.models import vgg16
vgg_model = vgg16(pretrained=True)
summary(vgg_model, input_size=(3,224,224))

/home/dell/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       1,792
|    └─ReLU: 2-2                         --
|    └─Conv2d: 2-3                       36,928
|    └─ReLU: 2-4                         --
|    └─MaxPool2d: 2-5                    --
|    └─Conv2d: 2-6                       73,856
|    └─ReLU: 2-7                         --
|    └─Conv2d: 2-8                       147,584
|    └─ReLU: 2-9                         --
|    └─MaxPool2d: 2-10                   --
|    └─Conv2d: 2-11                      295,168
|    └─ReLU: 2-12                        --
|    └─Conv2d: 2-13                      590,080
|    └─ReLU: 2-14                        --
|    └─Conv2d: 2-15                      590,080
|    └─ReLU: 2-16                        --
|    └─MaxPool2d: 2-17                   --
|    └─Conv2d: 2-18                      1,180,160
|    └─ReLU: 2-19                        --
|    └─Conv2d: 2-20              

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       1,792
|    └─ReLU: 2-2                         --
|    └─Conv2d: 2-3                       36,928
|    └─ReLU: 2-4                         --
|    └─MaxPool2d: 2-5                    --
|    └─Conv2d: 2-6                       73,856
|    └─ReLU: 2-7                         --
|    └─Conv2d: 2-8                       147,584
|    └─ReLU: 2-9                         --
|    └─MaxPool2d: 2-10                   --
|    └─Conv2d: 2-11                      295,168
|    └─ReLU: 2-12                        --
|    └─Conv2d: 2-13                      590,080
|    └─ReLU: 2-14                        --
|    └─Conv2d: 2-15                      590,080
|    └─ReLU: 2-16                        --
|    └─MaxPool2d: 2-17                   --
|    └─Conv2d: 2-18                      1,180,160
|    └─ReLU: 2-19                        --
|    └─Conv2d: 2-20              

In [43]:
print(vgg_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [44]:
for param in model.parameters():
    param.requires_grad = False

In [45]:
import torch
num_features = vgg_model.classifier[-1].in_features
print(num_features)
layers = list(vgg_model.classifier.children())[:-1]
print(layers)
layers.extend([torch.nn.Linear(num_features, 10)])
print(layers)
vgg_model.classifier = torch.nn.Sequential(*layers)

4096
[Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False)]
[Linear(in_features=25088, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=4096, bias=True), ReLU(inplace=True), Dropout(p=0.5, inplace=False), Linear(in_features=4096, out_features=10, bias=True)]


In [46]:
print(vgg_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [47]:
criterion = nn.CrossEntropyLoss()

In [48]:
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [49]:
epochs = 10
for i in range(epochs):
    train_loss = 0.0
    val_loss = 0.0
    vgg_model.train()
    for images, labels in train_generator:
        optimizer.zero_grad()
        pred = vgg_model(images)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
        print(train_loss)
    vgg_model.eval()
    for images, labels in val_generator:
        optimizer.zero_grad()
        pred = vgg_model(images)
        loss = criterion(pred, labels)
        val_loss += loss.item()*images.size(0)
train_loss = train_loss/len(train_generator)
val_loss = val_loss/len(val_generator)
print(train_loss)
print(val_loss)

22.433311939239502
44.17114734649658
67.85694122314453
91.48366212844849
22.06047773361206
46.112399101257324
68.12499046325684
91.88085556030273
21.800673007965088
45.304930210113525
69.0407657623291
91.64718627929688
23.44935894012451
44.86655235290527
67.98864841461182
91.69105291366577
22.262113094329834
44.308719635009766
68.19425106048584
87.5666606426239
22.56148099899292
44.07512187957764
66.82673692703247
90.442054271698
21.973888874053955
45.16244173049927
68.52227687835693
92.36356496810913
23.141815662384033
48.74911546707153
72.31582880020142
96.47953987121582
23.722898960113525
47.21269130706787
70.84500551223755
92.93174505233765
22.74005889892578
47.458298206329346
71.10028982162476
94.73446607589722
23.683616518974304
10.925445556640625


In [50]:
test_accuracy = 0.0
for images, labels in test_generator:
    output = model(images)
    #rint(output)
    _, pred = torch.max(output,1)
    #print(list(pred))
    #print(labels)
    num_correct = 0
    accuracy = 0.0
    for i in range(len(pred)):
        if pred[i]==labels[i]:
            num_correct+=1
    accuracy = (num_correct/images.size(0))*100
    #print(accuracy)
    test_accuracy += accuracy
#print(test_accuracy)
print(test_accuracy/len(test_generator))

20.0
